# MalDroid Data Ingest
This notebook handles the retrieval of MalDroid malware sample analysis (sample_for_analysis.apk.json) files from the MalDroid repo (http://205.174.165.80/CICDataset/MalDroid-2020/Dataset/Capturing_logs/). 
## Process
1. Take the URIs of the .tar.gz files for each sample from MalDroid_ref_raw.csv, extract them to .tar and then to a directory *malware class*/*hash*
2. Check to ensure the sample_for_analysis.apk.json file does not throw an error when opening. If yes, directory will be deleted and script will move to next sample
3. Move the sample_for_analysis.apk.json file from the *malware class*/*hash*/sample_for_analysis.apk subdirectory to under *malware class*/*hash*/ and delete the subdirectory

In [1]:
import pandas as pd 
import tarfile
import os
import urllib.request
import json
import shutil
import numpy as np

In [2]:
with open('MalDroid_ref_raw.csv') as ref_raw_path:
    ref_raw = pd.read_csv(ref_raw_path)
ref_raw = ref_raw[['hash', 'URI', 'malware_class']]
ref_raw = ref_raw.drop([13076])
# Drops final row; does not contain a sample due to parsing issues

In [3]:
issue_hashes = ['0ac36a24aa3dd7e8bd7f009ab6115481812ed053c8b81967b0796a8d2f098b73', '0c41904e61ca30362129e4b7f460b2d8b5b4319de24f497b0dbdfbd64ec0f87a', '0cceea84886d862a703264074ace86a76ec8218c10789929b28352feab0d3267', '0ec63150ec195601eaaf3a42330fe03f9ea97b123ccbc728ef883271c41f8dc3', '00847524ec1e69b2cdd53205cd9725295e87094eeed1567b3efb12191ded24d2', '0669dccf3a9a08ba5d148aed4568c6a15f65075d913243f160b4c5c5ab31fc61', '082b102aaab4354b7b003a6fe5b21274582b0ed49441351a4a3f8a23012b55a5', '1d06256e8e1b7b90b242bebc323a5ef2bbbc71e72185216c3f39462c4bcb71c7', '1da5fadcaa815d9619353b8bf1bf4646c47eabfbc154d5dc7941adc1795b04b6', '2BB5E30C62E2049ADE758CF0375367B02732CE24B4C56BB234B4C4DBA3760089', '2c9fd7881e1822b92639ef3e011e64d87784d23ac05876d3fd1cec0238d1c46d', '2d62d2e6c3ec431d36ac449c0c062eee6d98b3cdac246316bb2d2622a464b26c', '2d351ee5139813d65b0f10c6fb40f9786e6ca91163e3dd2150ad4ded51ce4082', '3acbca1fb0e82db03483c8e0fbb3978c287444162e7278d2c65ea2633067087a', '3c50d8354bf9f52f3b14930599ed5a59152d83772a9b8e4377c8e0d1b73121f7', '3e0b445a1d2e33997ce78f94d62f01b5bccd6dd2b351336e567a6aac370a0b7f','3fcf12e1546a2f6b494ad42b6b699e27c4501ee662363d39ee41fd7559ac7b80','4a595d6ad083a662076b3907bcc2f5c508d63e2a7d3c0c12b4ccab98870557fd','4ab2d0bee49938748d4abf357ed2185b19dc2984fe679c0c162b0064ccb490f3','4b904ba542637bade7ff105c4e6e617b57cb1e5fd70f586f439794424f52ba40','4ce2c46fd9d88b2a2552eda6f4afc4c7e19043909fd92a294dcad198ff85cbc4']
large_hashes = ['0ccf9cfae60602b20c679ad8d56cc14f94ebf50aed389ef9db8a86e717ed76ea', '0f18d0cf80f7619d17142110eadbfe1ad01bc5936f275d666c454234241ca382', '03c78d58f22b4ebe6bb92735d19c869e57d10ed3071644c1a429170e7c9f3d51', '048449f839bfa7afc61aed035ba870aff254b7534c8444cbb701e31ec6d47f24', '09c70721776a3d0b345d2fbc647b363648c91b2f8f3dd0c25cf928aba4ade179',
'1a2a6761a1279bfe9a352381dd66614b466b19aefaf17c300536be8e4be0b0c5',
'1a2b969ba56b3d3473fe45f87a04fa9610293e2ccf687b2d1d71220b8c765077',
'1a9a75579366f1d2dbeaa6d2423819d874537973815778eae3debef47da08a1d',
'1c31d50db2aca2c03836e7e0cbef657e97ee8dae70758c58f89975a11fc0b6ca', '1e83a89395c3de4d2f0945e21d3fd6a7324538c0aa613836cfbe212c2bc1d145', '2de4216275ee614c108ab69519e97c7a9fcf50b9593557d554749165ee55a4b5', '2fbbe31be65aecfc88bf9f321f128f28280ae7dc26117a0ac2c179d9c7521d94']
last_hash = '5ACE49F9F1A70B915A7E469289558C303F51CEEABF9ED6BDB5DC6818743A64CE'
last_hash = last_hash.lower()

# prev_index = ref_raw['hash'].loc[lambda x: x==last_hash].index
ref_raw = ref_raw.iloc[235:]

def ingest_loop(sample):
    last_hash = sample[0]
    if '.tar.gz' in sample[1]:
        base_path = sample[2].lower() + '/' + sample[0]
        temp_file = urllib.request.urlretrieve(sample[1], filename=None)[0]
        try:
            file = tarfile.open(temp_file)
            file.extractall(base_path)
            file.close()
        except:
            print('Error extracting on hash: ' + sample[0])
            issue_hashes.append(sample[0])
            try:
                shutil.rmtree(base_path)
            except:
                pass
            return
        default_path = open(base_path + '/sample_for_analysis.apk/sample_for_analysis.apk.json')
        try:
            json.load(default_path)
            default_path.close()
        except:
            print('Error loading JSON on hash: ' + sample[0])
            issue_hashes.append(sample[0])
            default_path.close()
            return
        json_size = os.stat(base_path + '/sample_for_analysis.apk/sample_for_analysis.apk.json').st_size
        if json_size >= 100000000:
            # GitHub will not host files larger than 100 MB, this removes and logs these files for compatability
            print('JSON file too large on hash: ' + sample[0])
            large_hashes.append(sample[0])
            shutil.rmtree(base_path)
            return
        shutil.move(base_path + '/sample_for_analysis.apk/sample_for_analysis.apk.json', base_path + '/sample_for_analysis.apk.json')
        shutil.rmtree(base_path + '/sample_for_analysis.apk')

ref_raw.apply(lambda x: ingest_loop(x.to_numpy()), axis = 1)

Error extracting on hash: 5ace49f9f1a70b915a7e469289558c303f51ceeabf9ed6bdb5dc6818743a64ce
Error extracting on hash: 5ad048ce9aeabd62088de2e8038f1540762049d64749e3a953928c2dc88e7922
Error extracting on hash: 5b741e527f86eca88b64f9f64691e903110f15db77b3947a067ee1a2c44e6403
JSON file too large on hash: 5b42432aad27746f04353fdd70bd8c2d979303eaf367de28d88f6b0cb72dd9a5
JSON file too large on hash: 5bc23138fd301b7f293f9b4efb13d72019815d64695a6f9b694c23c33f1442b8
Error extracting on hash: 5cbdc05183fb0ce11248ead0c4a5de031cc724171360b38214a4b3facf58ce7d
Error extracting on hash: 6a59cdfefdc77efe385002e13b4dfb0595020c506c06fb0261e0be911479792e
Error extracting on hash: 6a37609938daf3c7d139ef7f4222024054e18466ad752e771b26b01d36e6d8fb
Error extracting on hash: 6b869cb1e71098dc70359ef3770dd0394979d50faf0ef7f955a9b83958d472db
Error extracting on hash: 6cae32799b7ef805a72c575a2473d03488e2daaac7ea6906237bbd1ed74f2b55
Error extracting on hash: 6dded5a4e6457fee955020998ccf176f7175d016d8f504aff32e798ec5

RemoteDisconnected: Remote end closed connection without response

In [ ]:
issue_df = pd.DataFrame(issue_hashes, columns = ['hash'])
with open('issue_hashes.csv', 'w') as issue_write:
    issue_df.to_csv(issue_write)

large_df = pd.DataFrame(large_hashes, columns = ['hash'])
with open('large_hashes.csv', 'w') as large_write:
    large_df.to_csv(large_write)

with open('last_hash_processed.txt', 'w') as resume_write:
    resume_write.write(last_hash)